In [1]:
import tensorflow as tf
import time
import csv
import numpy as np

from collections import namedtuple, Counter
from tensorflow.contrib.tensorboard.plugins import projector

/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


**Process the data**

In [13]:
def preprocess(dataset_path, lyric_len):
    labels = []
    data = []
    tags = ['<START>', '<END>', '<B>' <'UKN>', '<PAD>'] 
    
    with open(dataset_path, 'r') as file:
        reader = csv.reader(file)
        count = 0
        new_song = 1
        for row in reader:
            if count > 0:
                label = ['<START>'] 
                for word in row[1].split(' '):
                    label += [word]
                if len(label) >= hps_dict['title_dec_steps']:
                    label = label[:hps_dict['title_dec_steps']-1]
                label += ['<END>']
                while len(label) < hps_dict['title_dec_steps']:
                    label += ['<PAD>']
                labels += [label]
                current_lyric = ['<START>']
                for line in row[2].split('\n'):
                    for word in line.split(' '):
                        if word is not "":
                            current_lyric += [word]
                    current_lyric += ['<B>']
                if len(current_lyric) >= lyric_len:
                    current_lyric = current_lyric[:lyric_len-1]
                current_lyric += ['<END>']
                while len(current_lyric) < lyric_len:
                    current_lyric += ['<PAD>']
                data += [current_lyric]
            count += 1    
    return labels,data


def words_frequency(words_1d):
    vocabulary_size=20000
    count=[['<UNK>',-1]]

    counter = Counter(words_1d).most_common(vocabulary_size - 1)
    count.extend(counter)
    dictionary={}
    for word,_ in count:
        dictionary[word]=len(dictionary)

    data=[]
    unk_count=0
    for word in words_1d:
        if word in dictionary:
            index=dictionary[word]
        else:
            index=0
            unk_count+=1
        data.append(index)
    count[0] = ('<UNK>', unk_count)
    reverse_dictionary=dict(zip(dictionary.values(),dictionary.keys()))
    return data,count,dictionary,reverse_dictionary

In [19]:
class TitleGenerator(object):
    """
        generating the title based on the lyrics
    """
    
    def __init__(self, hps, vocab):
        self._vocab = vocab
        self._hps = hps
        
    def _add_placeholders(self):
        
        hps = self._hps
        self._lyrics_batch = tf.placeholder(tf.int32,
                                            [hps.batch_size, hps.lyrics_enc_steps],
                                            name='lyrics_batch')
        self._titles_batch = tf.placeholder(tf.int32,
                                            [hps.batch_size, hps.title_dec_steps],
                                            name='titles_batch')
        
        self._titles_mask = tf.placeholder(tf.float32,
                                            [hps.batch_size, hps.title_dec_steps],
                                            name='titles_mask')
    
    def _make_feed_dict(self, batch):
        
        feed_dict = {}
        
        feed_dict[self._lyrics_batch] = batch.lyrics
        feed_dict[self._titles_batch] = batch.titles
        feed_dict[self._titles_mask] = batch.titles_mask
        
        return feed_dict
    
    def _add_encoder(self, encoder_inputs):
        
        hps = self._hps
        
        with tf.variable_scope('encoder') as encoder:
            if hps.cell_type == 'lstm':
                cell = tf.nn.rnn_cell.LSTMCell(
                    hps.hidden_dim,
                    initializer=self._rand_unif_init,
                    state_is_tuple=True)
            elif hps.cell_type == 'gru':
                cell = tf.nn.rnn_cell.GRUCell(
                    hps.hidden_dim,
                    kernel_initializer=self._rand_unif_init)
            else:
                raise ValueError("cell_type is not assigned")
            outputs, state = tf.nn.dynamic_rnn(cell, encoder_inputs,
                                               swap_memory=True,
                                               dtype=tf.float32,
                                               )
            encoder_outputs = tf.concat(axis=2, values=outputs)
            
        return encoder_outputs, state
    
    def _add_decoder(self, decoder_inputs, enc_state):
        
        hps = self._hps
        
        with tf.variable_scope('decoder'):
            if hps.cell_type == 'lstm':
                cell = tf.nn.rnn_cell.LSTMCell(
                    hps.hidden_dim,
                    state_is_tuple=True,
                    nitializer=self._rand_unif_init)
            elif hps.cell_type == 'gru':
                cell = tf.nn.rnn_cell.GRUCell(
                    hps.hidden_dim,
                    kernel_initializer=self._rand_unif_init)
            else:
                raise ValueError("cell_type is not assigned")

            outputs, state = tf.nn.dynamic_rnn(cell, decoder_inputs,
                                               dtype=tf.float32,
                                               initial_state=enc_state)
            
        return outputs, state
    
    def _add_seq2seq(self):
        
        hps = self._hps
        
        vsize = len(self._vocab)
        
        with tf.variable_scope('seq2seq'):
            
            self._rand_unif_init = tf.random_uniform_initializer(
                -hps.rand_unif_init_mag, hps.rand_unif_init_mag, seed=123)
            self._trunc_norm_init = tf.truncated_normal_initializer(
                stddev=hps.trunc_norm_init_std)
            
            with tf.variable_scope('embedding'):
                embedding = tf.get_variable(
                    'embedding',
                    [vsize, hps.emb_dim],
                    dtype = tf.float32,
                    initializer=self._trunc_norm_init)
#                 if hps.mode == 'train':
#                     self._add_emb_vis(embedding)
                lyrics_emb = tf.nn.embedding_lookup(embedding,
                                                   self._lyrics_batch)
                titles_emb = tf.nn.embedding_lookup(embedding,
                                                   self._titles_batch)
            _, enc_state = self._add_encoder(lyrics_emb)
            dec_outputs, _ = self._add_decoder(titles_emb,
                                               enc_state)
            
            with tf.variable_scope('output_projection') as projection:
                
                titles_vocab_scores = []
                
                w = tf.get_variable('w', [hps.hidden_dim, vsize],
                                    dtype=tf.float32,
                                    initializer=self._trunc_norm_init)
                v = tf.get_variable('v', [vsize], dtype=tf.float32,
                                    initializer=self._trunc_norm_init)
                title_vocab_scores = []
                dec_outputs_list = tf.unstack(dec_outputs, axis=1)
                for i, output in enumerate(dec_outputs_list):
                    if i > 0:
                        tf.get_variable_scope().reuse_variables()
                    titles_vocab_scores.append(tf.nn.xw_plus_b(output, w, v))
            titles_vocab_scores_stack = tf.stack(titles_vocab_scores, axis=1)
            self.titles_dec_ids = tf.argmax(titles_vocab_scores_stack, axis = -1)
            print(titles_vocab_scores_stack.get_shape())
            print(self._titles_batch.get_shape())
            self._loss = tf.contrib.seq2seq.sequence_loss(titles_vocab_scores_stack,
                                                         self._titles_batch,
                                                         self._titles_mask)
            print(self._loss)
            tf.summary.scalar('loss', self._loss)
    def _add_train_op(self):
        loss_to_minimize = self._loss
        tvars = tf.trainable_variables()
        gradients = tf.gradients(
            loss_to_minimize,
            tvars,
            aggregation_method=tf.AggregationMethod.EXPERIMENTAL_TREE)
        
        with tf.device("/gpu:0"):
            #TODO: add into hps
            grads, global_norm = tf.clip_by_global_norm(
                gradients, self._hps.max_grad_norm)
        tf.summary.scalar('global_norm', global_norm)
        #TODO: add into hps
        optimizer = tf.train.AdamOptimizer(self._hps.lr)
        with tf.device("/gpu:0"):
            self._train_op = optimizer.apply_gradients(
                zip(grads, tvars),
                global_step=self.global_step, name='train_step')
            
    def build_graph(self):
        tf.logging.info('Building graph...')
        t0 = time.time()
        self._add_placeholders()
        with tf.device("/gpu:0"):
            self._add_seq2seq()
        self.global_step = tf.Variable(0, name='global_step', trainable=False)
        if self._hps.mode == 'train':
            self._add_train_op()
        self._summaries = tf.summary.merge_all()
        t1 = time.time()
        tf.logging.info("Time to build graph: %i seconds", t1 - t0)
        
    def run_train_step(self, sess, batch):
    
        feed_dict = self._make_feed_dict(batch)
        
        to_return = {
            'train_op' : self._train_op,
            'summaries' : self._summaries,
            'loss' : self._loss,
            'global_step' : self.global_step,
            'title_ids': self.titles_dec_ids
        }
        
        return sess.run(to_return, feed_dict)
    
    def run_eval_step(self, sess, batch):

        feed_dict = self._make_feed_dict(batch)
        to_return = {
            'summaries': self._summaries,
            'loss': self._loss,
            'global_step': self.global_step,
            'title_ids': self.titles_dec_ids
        }

        return sess.run(to_return, feed_dict)
    
    def run_eval_step(self, sess, batch):

        feed_dict = self._make_feed_dict(batch)
        to_return = {
            'summaries': self._summaries,
            'loss': self._loss,
            'title_ids': self.titles_dec_ids
        }

        return sess.run(to_return, feed_dict)

In [4]:
class Batch(object):
    
    def __init__(self, lyrics, lyrics_mask, titles, titles_mask):
        self.lyrics = np.array(lyrics)
        self.lyrics_mask = np.array(lyrics_mask)
        self.titles = np.array(titles)
        self.titles_mask = np.array(titles_mask)

class Batcher(object):
    
    def __init__(self, hps, data):
        self._hps = hps
        self._data = data
        
    def next_batch(self):
        
        hps = self._hps
        
        data_size = len(self._data)
        
        batch_index = np.random.randint(data_size,
                                        size=hps.batch_size)
        
        # TODO, depends on the data structure
        titles = []
        lyrics = []
        titles_mask = []
        lyrics_mask = []
        for idx in batch_index:
            titles.append(self._data[idx][0])
            lyrics.append(self._data[idx][1])
            titles_mask.append(self._data[idx][2])
            lyrics_mask.append(self._data[idx][3])
        
        batch = Batch(lyrics, lyrics_mask, titles, titles_mask)
        
        return batch

In [79]:
def run_training(model, batcher, sess_context_manager,
                 sv, summary_writer, vocab):
    tf.logging.info("Starting run training")
    with sess_context_manager as sess:
        while True:
            batch = batcher.next_batch()
            tf.logging.info('Running training step...')
            t0 = time.time()
            result = model.run_train_step(sess, batch)
            t1 = time.time()
            tf.logging.info('seconds for training step: %.3f', t1 - t0)

            loss = result['loss']
            tf.logging.info('loss: %f', loss)

            title_ids = result['title_ids']
            summaries = result['summaries']
            train_step = result['global_step']
        
#         summary_writer.add_summary(summaries, train_step)
#         if train_step % 100 == 0:
#             summary_writer.flush()
            
def get_config():
    """Returns config for tf.session"""
    config = tf.ConfigProto(allow_soft_placement=True)
    config.gpu_options.allow_growth=True
    return config
        
def setup_training(model, batcher, vocab):
    #TODO
    model.build_graph()
    saver = tf.train.Saver(max_to_keep=3)
    sv = tf.train.Supervisor(logdir=hps.train_dir,
                         is_chief=True,
                         saver=saver,
                         summary_op=None,
                         save_summaries_secs=60,
                         save_model_secs=60,
                         global_step=model.global_step)
    summary_writer = sv.summary_writer
    tf.logging.info('Preparing or waiting for session...')
    sess_context_manager = sv.prepare_or_wait_for_session(config=get_config())
    tf.logging.info('Created session')
    try:
        run_training(model, batcher, sess_context_manager, sv, summary_writer, vocab)
    except KeyboardInterrupt:
        tf.logging.info('Caught keyboard interrupt or worker. ' + \
                        'Stopping supervisor')
        sv.stop()
        
def run_decode(model, hps, batcher, vocab):
    """
      Run the model with the best parameter, to evaluate the model and get ROUGE
      BLEU scores.
    """

    model.build_graph()
    saver = tf.train.Saver()
    sess = tf.Session(config=get_config())
    ckpt = tf.train.get_checkpoint_state(hps.ckpt_file)
    if ckpt and ckpt.model_checkpoint_path:
        saver.restore(sess, ckpt.model_checkpoint_path)
    else:
        ValueError("check point doesn't exist")

    file_path_dict = {}
    hypo_dict = {}
    ref_dict = {}

    for _ in range(20):
        batch = batcher.next_batch()
        # if the next batch is None, then stop the evluation.
        if not batch:
            break
        t0=time.time()
        result = model.run_eval_step(sess, batch)
#         lyrics = [reverse_vocab batch.lyrics[0]]
        title_dec_ids = result['title_ids']
        example = title_dec_ids[0]
        sentences = [reverse_vocab[_] for _ in example]
        print(sentences)
        t1=time.time()
        tf.logging.info('seconds for batch: %.2f', t1-t0)

def main(data):
    tf.logging.set_verbosity(tf.logging.INFO)
    tf.logging.info('Starting seq2seq in %s mode...', (hps.mode))
    batcher = Batcher(hps, data)
    
    tf.set_random_seed(111)
    
    #TODO: initialize the vocab
    
    if hps.mode == 'train':
        print("Creating model for training...")
        model = TitleGenerator(hps, vocab)
        setup_training(model, batcher, vocab)
    elif hps.mode == 'test':
        print("Creating model...")
        model = TitleGenerator(hps, vocab)
        run_decode(model, hps, batcher, vocab)
    else:
        raise ValueError("Lack of 'mode' flag, must be one of train/eval")

In [14]:
data_path = 'dataset.csv'
titles, lyrics = preprocess(data_path, hps.lyrics_enc_steps)

all_words = []
for t in titles:
    [all_words.append(_) for _ in t]
for l in lyrics:
    [all_words.append(_) for _ in l]
    
_, count, vocab, reverse_vocab = words_frequency(all_words)

In [15]:
data = []
for t, l in zip(titles, lyrics):
    t_num = [vocab[_] if _ in vocab.keys() else vocab['<UNK>'] for _ in t]
    l_num = [vocab[_] if _ in vocab.keys() else vocab['<UNK>']for _ in l]
    t_mask = [1 if _ is not '<PAD>' else 0 for _ in t]
    l_mask = [1 if _ is not '<PAD>' else 0 for _ in l]
    data.append([t_num, l_num, t_mask, l_mask])

In [80]:
hps_dict = {
    'mode': 'test',
    'batch_size': 32,
    'hidden_dim': 512,
    'emb_dim': 128,
    'cell_type': 'gru',
    'lyrics_enc_steps': 200,
    'title_dec_steps': 10,
    'rand_unif_init_mag': 0.1,
    'trunc_norm_init_std': 0.01,
    'max_grad_norm': 2,
    'train_dir':'./',
    'lr':0.001,
    'ckpt_file': './'
}

hps = namedtuple("HParams", hps_dict.keys())(**hps_dict)

tf.reset_default_graph()
main(data)

INFO:tensorflow:Starting seq2seq in test mode...
Creating model...
INFO:tensorflow:Building graph...
(32, 10, 20000)
(32, 10)
Tensor("seq2seq/sequence_loss/truediv:0", shape=(), dtype=float32, device=/device:GPU:0)
INFO:tensorflow:Time to build graph: 0 seconds
INFO:tensorflow:Restoring parameters from ./model.ckpt-220
['<START>', 'How', '<END>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>']
INFO:tensorflow:seconds for batch: 0.18
['<START>', 'How', 'Trick', '<UNK>', '<END>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>']
INFO:tensorflow:seconds for batch: 0.14
['<START>', 'I', 'You', 'See', 'The', 'World', '<END>', '<PAD>', '<PAD>', '<PAD>']
INFO:tensorflow:seconds for batch: 0.13
['<START>', 'You', '<END>', '<END>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>']
INFO:tensorflow:seconds for batch: 0.13
['<START>', 'How', '<UNK>', '<END>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>']
INFO:tensorflow:seconds for batch: 0.13
['<START>', 'Blue', 'Trick', 'Yo

In [ ]:
ckpt_state = tf.train.get_checkpoint_state(hps.train_dir, latest_filename='checkpoint')
ckpt_state.model_checkpoint_path